In [304]:
# Importing necessary libraries
import pandas as pd
import ast
import numpy as np
import random
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [260]:
product_catalog_df =pd.read_csv ("C:/data/Work/Data_processing/Feature_Engineering/Feature_product_catalog.csv")

In [261]:
# Display basic information about the dataset
product_info = {
    "Shape": product_catalog_df.shape,
    "Columns": list(product_catalog_df.columns),
    "Missing Values": product_catalog_df.isnull().sum().to_dict(),
    "Sample Data": product_catalog_df.head()}
print(product_info)

{'Shape': (100, 25), 'Columns': ['Product ID', 'Product Name', 'Category', 'Material', 'Size', 'Special_features', 'Customer Type', 'Price', 'Payment Type', 'Sellable Online', 'Sales Volume', 'Return Rate', 'Storage Cost', 'Seasonality Score', 'Implicit Feedback', 'Sales_to_Return_Ratio', 'Revenue_Per_Product', 'Online_Sellability_Score', 'Storage_Efficiency_Score', 'High_Demand_Indicator', 'Implicit_Feedback_Score', 'Top_Rated', 'Likelihood_of_Purchase', 'Customer_Interest_Score', 'Expert_Judgment_Score'], 'Missing Values': {'Product ID': 0, 'Product Name': 0, 'Category': 0, 'Material': 0, 'Size': 0, 'Special_features': 0, 'Customer Type': 0, 'Price': 0, 'Payment Type': 0, 'Sellable Online': 0, 'Sales Volume': 0, 'Return Rate': 0, 'Storage Cost': 0, 'Seasonality Score': 0, 'Implicit Feedback': 0, 'Sales_to_Return_Ratio': 0, 'Revenue_Per_Product': 0, 'Online_Sellability_Score': 0, 'Storage_Efficiency_Score': 0, 'High_Demand_Indicator': 0, 'Implicit_Feedback_Score': 0, 'Top_Rated': 0, '

In [262]:
# Select relevant columns for recommendations
selected_features = [
    "Product Name", "Category", "Material", "Size", "Special_features", 
    "Sales Volume", "Likelihood_of_Purchase", "Revenue_Per_Product", "Expert_Judgment_Score"
]

# Drop missing values
product_catalog_df = product_catalog_df.dropna(subset=selected_features)

# Normalize business-related numerical values using MinMaxScaler
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(product_catalog_df[["Sales Volume", "Likelihood_of_Purchase", "Revenue_Per_Product", "Expert_Judgment_Score"]])

# Add normalized values back to dataframe
product_catalog_df[["Sales_Score", "Purchase_Score", "Revenue_Score", "Expert_Score"]] = scaled_values

# Function to create feature strings with structured business metrics
def create_feature_string(row):
    return (
        f"{row['Category']} " * 4 + 
        f"{row['Material']} " * 3 +  
        f"{row['Special_features']} " * 2 +  
        f"{row['Size']} " * 1 +
        f"sales_{round(row['Sales_Score'], 2)} " + 
        f"purchase_{round(row['Purchase_Score'], 2)} " + 
        f"revenue_{round(row['Revenue_Score'], 2)} " + 
        f"expert_{round(row['Expert_Score'], 2)}"
    ).strip()

# Apply the function
product_catalog_df["Combined_Features"] = product_catalog_df.apply(create_feature_string, axis=1)

# Display sample feature combinations
print(product_catalog_df[["Product Name", "Combined_Features"]].head(10))

            Product Name  \
0   Kitchen Pantry Shelf   
1  Ottoman Storage Bench   
2       Rustic Bookshelf   
3        Rattan Armchair   
4     Glass Dining Table   
5   Upholstered armchair   
6      Wooden Side Table   
7      Smart Office Desk   
8          Bistro Tables   
9  Open Closet Organizer   

                                                                                                                                                                          Combined_Features  
0                               Kitchen & Dining Kitchen & Dining Kitchen & Dining Kitchen & Dining Wood Wood Wood Durable Durable Medium sales_0.21 purchase_0.21 revenue_0.12 expert_0.33  
1  Storage & Organization Storage & Organization Storage & Organization Storage & Organization Fabric Fabric Fabric Foldable Foldable Large sales_0.7 purchase_0.7 revenue_0.27 expert_0.67  
2        Storage & Organization Storage & Organization Storage & Organization Storage & Organization Wood Wood Wood Durab

In [263]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words="english", max_features=1000, ngram_range=(1,2))

# Convert Combined_Features column into a TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(product_catalog_df["Combined_Features"])

# Display matrix shape
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

TF-IDF Matrix Shape: (100, 536)


In [264]:
# Compute cosine similarity between product TF-IDF vectors
cosine_sim = cosine_similarity(tfidf_matrix)

# Display shape of similarity matrix
print("Cosine Similarity Matrix Shape:", cosine_sim.shape)

# Show a sample of the similarity matrix
print("Sample Similarity Scores:\n", np.round(cosine_sim[:5, :5], 2))


Cosine Similarity Matrix Shape: (100, 100)
Sample Similarity Scores:
 [[1.   0.01 0.11 0.07 0.05]
 [0.01 1.   0.57 0.01 0.02]
 [0.11 0.57 1.   0.05 0.08]
 [0.07 0.01 0.05 1.   0.04]
 [0.05 0.02 0.08 0.04 1.  ]]


In [265]:
def recommend_products(product_name, product_catalog_df, cosine_sim, top_n=5, min_similarity=0.1):
    print(f"\nSearching for recommendations for: {product_name}")
    
    # Find all indices of the selected product name
    product_indices = product_catalog_df[product_catalog_df["Product Name"] == product_name].index

    if product_indices.empty:
        print("Product not found in the dataset.")
        return [f"Product '{product_name}' not found in the catalog."]

    product_index = product_indices[0]
    similarity_scores = list(enumerate(cosine_sim[product_index]))

    # Sort by similarity and exclude itself
    sorted_products = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:]
    
    recommendations = []
    seen_products = set()

    for idx, score in sorted_products:
        rec_product = product_catalog_df.iloc[idx]

        # Print similarity scores for debugging
        print(f"Checking Product: {rec_product['Product Name']} - Score: {score:.2f}")

        # Only add unique product names, avoiding duplicates
        if rec_product["Product Name"] not in seen_products and score >= min_similarity:
            recommendations.append(rec_product["Product Name"])
            seen_products.add(rec_product["Product Name"])

        if len(recommendations) == top_n:
            break

    if not recommendations:
        print("No strong matches found.")   
    
    return recommendations if recommendations else ["No strong matches found. Try another product."]







In [266]:
# Use Case 
# Find similar products for a given product
product_to_search = "Rustic Bookshelf"  # Change this to a real product name from your dataset
recommendations = recommend_products(product_to_search, product_catalog_df, cosine_sim)

# Display recommendations
print(f"\nRecommended Products for '{product_to_search}':")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")


Searching for recommendations for: Rustic Bookshelf
Checking Product: Modern TV Cabinet - Score: 0.69
Checking Product: Sliding Door Wardrobe - Score: 0.69
Checking Product: Rustic Bookshelf - Score: 0.68
Checking Product: Sliding Door Wardrobe - Score: 0.65
Checking Product: Sliding Door Wardrobe - Score: 0.62
Checking Product: Modern TV Cabinet - Score: 0.62
Checking Product: Rustic Bookshelf - Score: 0.61
Checking Product: Rustic Bookshelf - Score: 0.60
Checking Product: Ottoman Storage Bench - Score: 0.60
Checking Product: Rustic Bookshelf - Score: 0.60
Checking Product: Sliding Door Wardrobe - Score: 0.58
Checking Product: Compact Shoe Rack - Score: 0.58

Recommended Products for 'Rustic Bookshelf':
1. Modern TV Cabinet
2. Sliding Door Wardrobe
3. Rustic Bookshelf
4. Ottoman Storage Bench
5. Compact Shoe Rack


In [246]:
#Checking the number of products in the dataset 
print(product_catalog_df["Product Name"].nunique()) 


30


In [ ]:
# Get all unique products from the product catalog
all_products = set(product_catalog_df["Product Name"].unique())

# Display total count
print(f" Total Products in Catalog: {len(all_products)}")


🔹 Total Products in Catalog: 30


In [287]:
#Collaborative Filtering
# Load the product catalog dataset
customer_survey_df = pd.read_csv("C:/data/Work/Data_processing/Feature_Engineering/Feature_customer_survey.csv")



In [298]:
#Rename "Suggested Products" to "Product Name" in the customer survey data
if "Suggested Products" in customer_survey_df.columns:
    customer_survey_df.rename(columns={"Suggested Products": "Product Name"}, inplace=True)

# Convert "Product Name" column from lists stored as strings to actual lists
if customer_survey_df["Product Name"].dtype == object:
    try:
        customer_survey_df["Product Name"] = customer_survey_df["Product Name"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("[") else [x]
        )
    except Exception as e:
        print(f"Error converting 'Product Name' column: {e}")

# Explode "Product Name" column to separate rows for each product
customer_survey_df = customer_survey_df.explode("Product Name")

# Standardize product names 
customer_survey_df["Product Name"] = customer_survey_df["Product Name"].str.lower().str.strip()
product_catalog_df["Product Name"] = product_catalog_df["Product Name"].str.lower().str.strip()


#  Find unmatched products 
survey_products = set(customer_survey_df["Product Name"])
catalog_products = set(product_catalog_df["Product Name"])
unmatched_products = survey_products - catalog_products

print(f" Products in Survey but NOT in Catalog: {len(unmatched_products)}")
print(list(unmatched_products))

#  Filter out unmatched products
customer_survey_df = customer_survey_df[customer_survey_df["Product Name"].isin(catalog_products)]

# Selecting relevant columns for collaborative filtering
customer_survey_df = customer_survey_df[["Customer ID", "Product Name", "Recommendation Score", 
                                         "Purchase Frequency", "Review_Sentiment", "Engagement_Sentiment"]]

product_catalog_df = product_catalog_df[["Product Name", "Implicit_Feedback_Score"]]

#  Merge datasets
merged_df = pd.merge(customer_survey_df, product_catalog_df, on="Product Name", how="inner")

#  Fill missing values 
merged_df.fillna(0, inplace=True)



 Products in Survey but NOT in Catalog: 0
[]


In [299]:
print(merged_df.head())


   Customer ID      Product Name  Recommendation Score Purchase Frequency  \
0            1  rustic bookshelf             -0.173299            monthly   
1            1  rustic bookshelf             -0.173299            monthly   
2            1  rustic bookshelf             -0.173299            monthly   
3            1  rustic bookshelf             -0.173299            monthly   
4            1  rustic bookshelf             -0.173299            monthly   

   Review_Sentiment  Engagement_Sentiment  Implicit_Feedback_Score  
0                 1                     1                        0  
1                 1                     1                        1  
2                 1                     1                       -1  
3                 1                     1                        0  
4                 1                     1                        0  


In [300]:
# Create Final Score (Combining Recommendation Score & Implicit Feedback Score)
merged_df["Final_Score"] = merged_df["Recommendation Score"].combine_first(merged_df["Implicit_Feedback_Score"])

# Create User-Product Interaction Matrix
interaction_matrix = merged_df.pivot_table(index="Customer ID",  # Rows = Customers
                                           columns="Product Name",  # Columns = Products
                                           values="Final_Score",  # Values = Scores
                                           aggfunc="mean")  # Average in case of duplicates

# Fill missing values 
interaction_matrix.fillna(0, inplace=True)

#  Display final interaction matrix
print("User-Product Interaction Matrix:")
print(interaction_matrix.head())

User-Product Interaction Matrix:
Product Name  6-piece outdoor dining set  adjustable standing desk  \
Customer ID                                                          
1                                    0.0                 -0.173299   
2                                    0.0                  0.000000   
3                                    0.0                  0.000000   
4                                    0.0                  0.000000   
5                                    0.0                 -0.699509   

Product Name  chaise sofas  compact shoe rack  foldable sun lounger  \
Customer ID                                                           
1                  0.00000                0.0                   0.0   
2                  0.00000                0.0                   0.0   
3                  0.00000                0.0                   0.0   
4                  0.24767                0.0                   0.0   
5                  0.00000                0.0     

In [302]:
# Extract all products from the catalog
all_products = set(product_catalog_df["Product Name"])

# Extract all products that have been interacted with in the interaction matrix
engaged_products = set(interaction_matrix.columns)

# Find the unused products (products in catalog but not in interaction matrix)
unused_products = all_products - engaged_products

# Display results
print(f"❌ Unengaged Products: {len(unused_products)}\n")
print("🛑 Products That Are Still Not Engaged:")
for product in sorted(unused_products):
    print("-", product)


❌ Unengaged Products: 14

🛑 Products That Are Still Not Engaged:
- adjustable bar stool
- bistro tables
- classic nightstand
- expandable kitchen island
- garden lounge bench
- glass dining table
- hanging hammock
- industrial console table
- king size bed frame
- kitchen pantry shelf
- large wooden dresser
- modern tv cabinet
- open closet organizer
- wooden side table


In [309]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Convert interaction matrix to binary (1 = interaction, 0 = no interaction)
binary_matrix = (interaction_matrix > 0).astype(int)

# Remove users with no interactions
binary_matrix = binary_matrix.loc[(binary_matrix.sum(axis=1) > 0), :]

# Compute cosine similarity
user_similarity_cosine = cosine_similarity(binary_matrix)

# Convert back to DataFrame
user_similarity_cosine_df = pd.DataFrame(
    user_similarity_cosine, 
    index=binary_matrix.index, 
    columns=binary_matrix.index
)

# Display sample output
print("User Similarity Matrix (Cosine, Sample):")
print(user_similarity_cosine_df.iloc[:5, :5])







User Similarity Matrix (Cosine, Sample):
Customer ID    2    4    7    8    9
Customer ID                         
2            1.0  0.0  0.0  0.0  0.0
4            0.0  1.0  0.0  0.0  0.0
7            0.0  0.0  1.0  0.0  0.0
8            0.0  0.0  0.0  1.0  0.0
9            0.0  0.0  0.0  0.0  1.0


In [310]:
# Count how many unique customers interacted with each product
product_interaction_counts = merged_df.groupby("Product Name")["Customer ID"].nunique().sort_values(ascending=False)

# Display top 10 most interacted products
print("🛍️ Top 10 Most Interacted Products:")
print(product_interaction_counts.head(10))

# Display bottom 10 least interacted products
print("\n🚨 Least Interacted Products:")
print(product_interaction_counts.tail(10))


🛍️ Top 10 Most Interacted Products:
Product Name
adjustable standing desk      19
smart office desk             15
chaise sofas                  13
padded dining chair           11
rustic bookshelf              11
foldable sun lounger          10
compact shoe rack              9
6-piece outdoor dining set     9
upholstered armchair           9
outdoor patio chair            9
Name: Customer ID, dtype: int64

🚨 Least Interacted Products:
Product Name
compact shoe rack             9
6-piece outdoor dining set    9
upholstered armchair          9
outdoor patio chair           9
luxury bean bag               9
sliding door wardrobe         8
ottoman storage bench         6
rattan armchair               5
minimalist sectional sofa     4
queen bed with storage        4
Name: Customer ID, dtype: int64


In [311]:
# Count how many unique products each customer has interacted with
customer_interaction_counts = merged_df.groupby("Customer ID")["Product Name"].nunique().sort_values(ascending=False)

# Display top 10 customers with most interactions
print("👤 Customers with Most Interactions:")
print(customer_interaction_counts.head(10))

# Display bottom 10 customers with the least interactions
print("\n⚠️ Customers with Least Interactions:")
print(customer_interaction_counts.tail(10))


👤 Customers with Most Interactions:
Customer ID
1     2
3     2
5     2
11    2
7     2
10    2
9     2
16    2
17    2
13    2
Name: Product Name, dtype: int64

⚠️ Customers with Least Interactions:
Customer ID
66    1
76    1
79    1
91    1
92    1
84    1
81    1
82    1
85    1
95    1
Name: Product Name, dtype: int64


In [312]:
# Find products bought by only one customer
single_buyer_products = product_interaction_counts[product_interaction_counts == 1]

print(f"🔍 Number of Products Bought by Only 1 Customer: {len(single_buyer_products)}")
print("\n🛑 Products Bought by Only One Customer:")
print(single_buyer_products.index.tolist())


🔍 Number of Products Bought by Only 1 Customer: 0

🛑 Products Bought by Only One Customer:
[]


In [313]:
# Find customers who bought only one product
single_product_customers = customer_interaction_counts[customer_interaction_counts == 1]

print(f"🔍 Number of Customers Who Bought Only 1 Product: {len(single_product_customers)}")
print("\n⚠️ Customers Who Bought Only One Product:")
print(single_product_customers.index.tolist())


🔍 Number of Customers Who Bought Only 1 Product: 31

⚠️ Customers Who Bought Only One Product:
[20, 22, 18, 15, 14, 6, 8, 2, 4, 23, 57, 40, 31, 37, 48, 62, 58, 74, 78, 73, 72, 66, 76, 79, 91, 92, 84, 81, 82, 85, 95]
